In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, f1_score

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

In [2]:
train_df = pd.read_csv('train.csv', usecols=list(range(1, 45)))
test_df = pd.read_csv('test.csv', usecols=list(range(1, 45)))

# Explore the datasets
print("Training Data Shape:", train_df.shape)
print("Test Data Shape:", test_df.shape)

Training Data Shape: (40000, 44)
Test Data Shape: (5000, 44)


In [3]:
print(train_df.isnull().sum())
print(test_df.isnull().sum())


policy_id                           0
policy_tenure                       0
age_of_car                          0
age_of_policyholder                 0
area_cluster                        0
population_density                  0
make                                0
segment                             0
model                               0
fuel_type                           0
max_torque                          0
max_power                           0
engine_type                         0
airbags                             0
is_esc                              0
is_adjustable_steering              0
is_tpms                             0
is_parking_sensors                  0
is_parking_camera                   0
rear_brakes_type                    0
displacement                        0
cylinder                            0
transmission_type                   0
gear_box                            0
steering_type                       0
turning_radius                      0
length      

In [4]:
train_df

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,P1,54,5 years and 3 months,37,C9,17804,3,C2,M4,Diesel,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
1,P2,36,8 months,42,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
2,P3,79,4 months,37,C5,34738,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
3,P4,46,0 months,37,C9,17804,1,B1,M8,CNG,...,No,Yes,Yes,Yes,No,No,Yes,Yes,2,0
4,P5,66,5 years and 3 months,36,C10,73430,3,C2,M4,Diesel,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,P39996,25,1 years and 0 months,34,C10,73430,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
39996,P39997,11,3 years and 1 months,30,C8,8794,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
39997,P39998,96,8 years and 0 months,44,C8,8794,3,C2,M4,Diesel,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
39998,P39999,61,2 years and 1 months,60,C7,6112,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0


In [5]:
test_df

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,P45001,46,0 months,50,C14,7788,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
1,P45002,95,4 months,25,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,1
2,P45003,45,1 years and 0 months,26,C2,27003,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
3,P45004,36,5 years and 11 months,34,C22,16733,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,P45005,91,1 years and 9 months,24,C2,27003,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,P49996,8,4 months,23,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
4996,P49997,9,0 months,33,C5,34738,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
4997,P49998,33,5 years and 7 months,47,C8,8794,3,C2,M4,Diesel,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
4998,P49999,68,1 years and 0 months,25,C8,8794,1,B1,M8,CNG,...,No,Yes,Yes,Yes,No,No,Yes,Yes,2,0


In [6]:
print(train_df.columns)

Index(['policy_id', 'policy_tenure', 'age_of_car', 'age_of_policyholder',
       'area_cluster', 'population_density', 'make', 'segment', 'model',
       'fuel_type', 'max_torque', 'max_power', 'engine_type', 'airbags',
       'is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors',
       'is_parking_camera', 'rear_brakes_type', 'displacement', 'cylinder',
       'transmission_type', 'gear_box', 'steering_type', 'turning_radius',
       'length', 'width', 'height', 'gross_weight', 'is_front_fog_lights',
       'is_rear_window_wiper', 'is_rear_window_washer',
       'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks',
       'is_central_locking', 'is_power_steering',
       'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror',
       'is_ecw', 'is_speed_alert', 'ncap_rating', 'is_claim'],
      dtype='object')


In [7]:
def convert_age_to_months(age_str):
    age_str = str(age_str)  # Ensure the input is treated as a string
    years = 0
    months = 0
    if 'years' not in age_str:
        years = 0
        months = int(age_str.split()[0])
    else:
        years = int(age_str.split()[0])
        months = int(age_str.split()[3])
    return years * 12 + months

def encode_ages(df):
    # Encode ages 23 to 78 as classes 0 to 55
    df['age_class'] = df['age_of_policyholder'] - 23
    return df

In [8]:
def process_data_fortrain(df):
    # Create a copy of the DataFrame to avoid modifying the original data
    df_copy = df.copy()

    # Binary feature conversion
    binary_features = [col for col in df_copy.columns if df_copy[col].dropna().isin(['Yes', 'No']).all()]
    for col in binary_features:
        df_copy[col] = df_copy[col].map({'Yes': 1, 'No': 0})

    # List of numeric features for normalization
    numeric_features = ['policy_tenure', 'population_density', 'airbags', 'displacement', 'cylinder', 'turning_radius', 'length', 'width', 'height', 'gross_weight']

    # for feature in numeric_features:
    #     Q1 = df_copy[feature].quantile(0.25)
    #     Q3 = df_copy[feature].quantile(0.75)
    #     IQR = Q3 - Q1
    #     lower_bound = Q1 - 1.5 * IQR
    #     upper_bound = Q3 + 1.5 * IQR
    #     df_copy = df_copy[(df_copy[feature] >= lower_bound) & (df_copy[feature] <= upper_bound)]
    # Categorical feature encoding
    categorical_features = ['area_cluster', 'segment', 'model', 'fuel_type', 'engine_type', 'rear_brakes_type', 'transmission_type', 'steering_type']
    for col in categorical_features:
        df_copy[col], _ = pd.factorize(df_copy[col])

    # Extract numeric values from 'max_torque' and 'max_power'
    df_copy['max_torque'] = df_copy['max_torque'].astype(str).str.extract(r'(\d+)').astype(float)
    df_copy['max_power'] = df_copy['max_power'].astype(str).str.extract(r'(\d+\.\d+|\d+)').astype(float)

    # Convert 'age_of_car' to total months using a previously defined function convert_age_to_months
    df_copy['age_of_car'] = df_copy['age_of_car'].apply(convert_age_to_months)

    # all_numeric = ['policy_tenure','age_of_car','max_power','max_torque', 'population_density', 'airbags', 'displacement', 'cylinder', 'turning_radius', 'length', 'width', 'height', 'gross_weight']

    # poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
    # poly_features = poly.fit_transform(df_copy[all_numeric])
    # poly_feature_names = poly.get_feature_names_out(all_numeric)  # Get the feature names for the polynomial features

    # # Create a DataFrame for the polynomial features
    # df_poly_features = pd.DataFrame(poly_features, columns=poly_feature_names, index=df_copy.index)
    
    # # Concatenate the polynomial features with the original dataframe
    # df_copy = pd.concat([df_copy, df_poly_features], axis=1)
    # Normalize numeric features
    df_copy[numeric_features] = (df_copy[numeric_features] - df_copy[numeric_features].mean()) / df_copy[numeric_features].std()

    return df_copy

In [9]:
def process_data_fortest(df):
    # Create a copy of the DataFrame to avoid modifying the original data
    df_copy = df.copy()

    # Binary feature conversion
    binary_features = [col for col in df_copy.columns if df_copy[col].dropna().isin(['Yes', 'No']).all()]
    for col in binary_features:
        df_copy[col] = df_copy[col].map({'Yes': 1, 'No': 0})

    # List of numeric features for normalization
    numeric_features = ['policy_tenure', 'population_density', 'airbags', 'displacement', 'cylinder', 'turning_radius', 'length', 'width', 'height', 'gross_weight']

    # Categorical feature encoding
    categorical_features = ['area_cluster', 'segment', 'model', 'fuel_type', 'engine_type', 'rear_brakes_type', 'transmission_type', 'steering_type']
    for col in categorical_features:
        df_copy[col], _ = pd.factorize(df_copy[col])

    # Extract numeric values from 'max_torque' and 'max_power'
    df_copy['max_torque'] = df_copy['max_torque'].astype(str).str.extract(r'(\d+)').astype(float)
    df_copy['max_power'] = df_copy['max_power'].astype(str).str.extract(r'(\d+\.\d+|\d+)').astype(float)

    # Convert 'age_of_car' to total months using a previously defined function convert_age_to_months
    df_copy['age_of_car'] = df_copy['age_of_car'].apply(convert_age_to_months)

    # all_numeric = ['policy_tenure','age_of_car','max_power','max_torque', 'population_density', 'airbags', 'displacement', 'cylinder', 'turning_radius', 'length', 'width', 'height', 'gross_weight']

    # poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
    # poly_features = poly.fit_transform(df_copy[all_numeric])
    # poly_feature_names = poly.get_feature_names_out(all_numeric)  # Get the feature names for the polynomial features

    # # Create a DataFrame for the polynomial features
    # df_poly_features = pd.DataFrame(poly_features, columns=poly_feature_names, index=df_copy.index)
    
    # # Concatenate the polynomial features with the original dataframe
    # df_copy = pd.concat([df_copy, df_poly_features], axis=1)

    # Normalize numeric features
    df_copy[numeric_features] = (df_copy[numeric_features] - df_copy[numeric_features].mean()) / df_copy[numeric_features].std()

    return df_copy

In [10]:
train_df = pd.read_csv('train.csv', usecols=list(range(1, 45)))
tmp_df = process_data_fortrain(train_df)

In [11]:
numeric_features = ['policy_tenure', 'population_density', 'airbags', 'displacement', 'cylinder', 'turning_radius', 'length', 'width', 'height', 'gross_weight']
tmp_df.columns

Index(['policy_id', 'policy_tenure', 'age_of_car', 'age_of_policyholder',
       'area_cluster', 'population_density', 'make', 'segment', 'model',
       'fuel_type', 'max_torque', 'max_power', 'engine_type', 'airbags',
       'is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors',
       'is_parking_camera', 'rear_brakes_type', 'displacement', 'cylinder',
       'transmission_type', 'gear_box', 'steering_type', 'turning_radius',
       'length', 'width', 'height', 'gross_weight', 'is_front_fog_lights',
       'is_rear_window_wiper', 'is_rear_window_washer',
       'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks',
       'is_central_locking', 'is_power_steering',
       'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror',
       'is_ecw', 'is_speed_alert', 'ncap_rating', 'is_claim'],
      dtype='object')

In [131]:
def train_and_evaluate(train_df, test_df):
    tmp_train_df = process_data_fortrain(train_df)
    tmp_test_df = process_data_fortrain(test_df)
    tmp_train_df = encode_ages(tmp_train_df)
    tmp_test_df = encode_ages(tmp_test_df)
    X = tmp_train_df.drop(['age_of_policyholder', 'is_claim', 'policy_id'], axis=1)
    X_test = tmp_test_df.drop(['age_of_policyholder', 'is_claim', 'policy_id'], axis=1, errors='ignore')
    # X = process_data_fortrain(X)
    # X_test = process_data_fortest(X_test)
    features = ['policy_tenure', 'age_of_car', 'area_cluster', 'population_density']

    # print(X)
    y_age = tmp_train_df['age_class']
    y_test = tmp_test_df['age_class']
    y_claim = tmp_train_df['is_claim']
    y_claim_test = tmp_test_df['is_claim']

    class_weights = {0: 1, 1: 10}

    

    models = {
        # 'Ridge': Ridge(alpha = 0.2),
        # 'Lasso': Lasso(),
        # 'ElasticNet': ElasticNet(),
        # 'DecisionTree': DecisionTreeRegressor(max_depth = 10,min_samples_split=50, min_samples_leaf=10),
        # 'RandomForest': RandomForestRegressor(n_estimators = 70, max_depth = 10,min_samples_split=50, min_samples_leaf=10),
        # 'mlp': MLPRegressor(hidden_layer_sizes=(400,100), max_iter=100, activation='relu', verbose=True)
        'RandomForest': RandomForestClassifier(n_estimators = 200, max_depth = 11),
        # 'GradientBoosting': GradientBoostingRegressor(n_estimators = 60, ma_depth = 10, learning_rate = 0.08, min_samples_split=55, min_samples_leaf=10)
   #n_estimators = 75~120, max_depth = 8-15, learning_rate = 0.07 - 0.15, min_samples_split=~55理解55正解, min_samples_leaf=10~20偏不重要
    }

    # param_grid = {
    # 'n_estimators': [40, 50, 70, 80, 100, 200, 300],
    # 'learning_rate': [0.001, 0.05, 0.01, 0.1, 0.2],
    # 'max_depth': [3, 4, 5, 6],
    # 'min_samples_split': [2, 4],
    # 'min_samples_leaf': [1, 2],
    # 'subsample': [0.8, 0.9, 1.0]
    # }
    
    # pca = PCA(n_components=40)
    # X = pca.fit_transform(X)
    # X_test = pca.transform(X_test)
    # print(X.shape)
    
  
    results = {}
    for name, model in models.items():
        # grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
        # grid_search.fit(X, y_age)
        model.fit(X,y_age)
        age_pred_train = model.predict(X)
        age_pred_test = model.predict(X_test)


        # Convert predictions back to ages
        age_pred_train = age_pred_train+23
        age_pred_test = age_pred_test+23
        age_pred_test = np.round(age_pred_test).astype(int)
        print(age_pred_train.max())
        print(age_pred_train.min())
        print(test_df['age_of_policyholder'])
        print(age_pred_test)
        print(age_pred_test.max())
        print(age_pred_test.min())
        mse_train = mean_squared_error(train_df['age_of_policyholder'], age_pred_train)
        mse_test = mean_squared_error(test_df['age_of_policyholder'], age_pred_test)
        results[name] = {'Train MSE': mse_train, 'Test MSE': mse_test}
        print(name)
        print(f"Train MSE: {mse_train}")
        print(f"Test MSE: {mse_test}")

    forest = RandomForestRegressor()
    forest.fit(X, y_age)
    importances = forest.feature_importances_
    # print(importances)
    feature_importances = pd.DataFrame({
        'feature': X.columns,   # Assuming X is a DataFrame
        'importance': importances
    })



    # Sort the DataFrame by importance
    feature_importances_sorted = feature_importances.sort_values(by='importance', ascending=False)
    top_features = feature_importances_sorted.head(10)['feature']
    X = X[top_features]
    X_test = X_test[top_features]
    
    rf_claim = RandomForestClassifier(class_weight=class_weights,max_depth = 7,random_state = 1)
    rf_claim.fit(X, y_claim)
    claim_pred_train =  rf_claim.predict(X)
    claim_pred_test =  rf_claim.predict(X_test) 
    f1_train = f1_score(y_claim, claim_pred_train, average='macro')
    f1_test = f1_score(y_claim_test, claim_pred_test, average='macro')
    print(f"Train Macro F1-Score: {f1_train}")
    print(f"Test Macro F1-Score: {f1_test}")

    return age_pred_test, claim_pred_test, test_df['policy_id'], mse_train, f1_train, mse_test, f1_test


In [132]:
def write_output(age_predictions, claim_predictions, policy_ids, id):
    part1_output = pd.DataFrame({
        'policy_id': policy_ids,
        'age': age_predictions
    })
    part1_output.to_csv(f'z{id}.PART1.output.csv', index=False)

    part2_output = pd.DataFrame({
        'policy_id': policy_ids,
        'is_claim': claim_predictions
    })
    part2_output.to_csv(f'z{id}.PART2.output.csv', index=False)


In [133]:
train_df

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,P1,54,5 years and 3 months,37,C9,17804,3,C2,M4,Diesel,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
1,P2,36,8 months,42,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
2,P3,79,4 months,37,C5,34738,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
3,P4,46,0 months,37,C9,17804,1,B1,M8,CNG,...,No,Yes,Yes,Yes,No,No,Yes,Yes,2,0
4,P5,66,5 years and 3 months,36,C10,73430,3,C2,M4,Diesel,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,P39996,25,1 years and 0 months,34,C10,73430,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
39996,P39997,11,3 years and 1 months,30,C8,8794,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
39997,P39998,96,8 years and 0 months,44,C8,8794,3,C2,M4,Diesel,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
39998,P39999,61,2 years and 1 months,60,C7,6112,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0


In [134]:
age_predictions, claim_predictions, policy_ids, mse_train, f1_train, mse_test, f1_test= train_and_evaluate(train_df, test_df)

71
23
0       50
1       25
2       26
3       34
4       24
        ..
4995    23
4996    33
4997    47
4998    25
4999    47
Name: age_of_policyholder, Length: 5000, dtype: int64
[46 26 26 ... 36 25 40]
53
23
RandomForest
Train MSE: 36.882775
Test MSE: 40.0702
Train Macro F1-Score: 0.5613171929194665
Test Macro F1-Score: 0.5220059669869959


In [118]:
train_df['age_of_policyholder'].max()

78

In [33]:
claim_counts = test_df['is_claim'].value_counts()

# Print the counts of each class
print(claim_counts)

is_claim
0    4681
1     319
Name: count, dtype: int64


In [34]:
numeric_features = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(numeric_features)

['policy_tenure', 'age_of_policyholder', 'population_density', 'make', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'length', 'width', 'height', 'gross_weight', 'ncap_rating', 'is_claim']


In [35]:
categorical_features = [col for col in categorical_features if col not in binary_features]

print(categorical_features)

NameError: name 'categorical_features' is not defined